<a href="https://colab.research.google.com/github/kidujm/data_class/blob/main/%EB%A9%94%EA%B0%80%EB%B0%95%EC%8A%A4_%EC%9C%A0%EC%A0%95%EB%AF%BC_0211.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 메가박스 리뷰 크롤링 정제
- 결측값,중복값 제거


In [ ]:
import pandas as pd

# 파일 경로 (환경에 맞게 수정 필요)
file_path = 'megabox_reviews.csv'

# CSV 파일 불러오기
df = pd.read_csv(file_path)

# 상위 5개 행 확인
print(df.head())


                               reviewId       userName  \
0  9d71356a-698e-447a-a60d-c5857bc5d175            정승렬   
1  20383b65-9489-4fb7-abd2-d826fee26c9b            윤은경   
2  8f350a8a-db12-417d-abb9-64e2ac7a6167            이준영   
3  284f8e28-ef0a-4f61-aa55-e8b1197f4b5f  김연오 (Kyo0103)   
4  07b953b1-f964-479b-a6a6-bfd47a7d79d6           김스데반   

                                           userImage  \
0  https://play-lh.googleusercontent.com/a-/ALV-U...   
1  https://play-lh.googleusercontent.com/a/ACg8oc...   
2  https://play-lh.googleusercontent.com/a-/ALV-U...   
3  https://play-lh.googleusercontent.com/a-/ALV-U...   
4  https://play-lh.googleusercontent.com/a/ACg8oc...   

                                        content  score  thumbsUpCount  \
0  결제버튼을눌러도 돈도안빠져나가고 예매도안돼요. 돈쓰지말라고 배려해주는건지....      2              0   
1                                자동로그인이 자꾸 풀리네요      1              1   
2                   오리지널 티켓 위탁점 배포 중단해서 이제 안가요.      1              0   
3                     

In [ ]:
# 각 컬럼별 결측값 개수 출력
missing_values = df.isnull().sum()
print("📌 컬럼별 결측값 개수:")
print(missing_values[missing_values > 0])

# 결측값 비율 포함해서 보기 좋게 출력
missing_info = pd.DataFrame({
    'Missing Count': df.isnull().sum(),
    'Missing Ratio (%)': df.isnull().mean() * 100
})
print("\n📌 결측값 요약:")
print(missing_info[missing_info['Missing Count'] > 0])


📌 컬럼별 결측값 개수:
reviewCreatedVersion     635
replyContent            4807
repliedAt               4807
appVersion               635
dtype: int64

📌 결측값 요약:
                      Missing Count  Missing Ratio (%)
reviewCreatedVersion            635          13.122546
replyContent                   4807          99.338706
repliedAt                      4807          99.338706
appVersion                      635          13.122546


In [ ]:
# 1. 'replyContent'와 'repliedAt' → '미답변'으로 채우기
df['replyContent'] = df['replyContent'].fillna('미답변')
df['repliedAt'] = df['repliedAt'].fillna('미답변')

# 2. 'reviewCreatedVersion', 'appVersion' → 'Unknown'으로 대체
df['reviewCreatedVersion'] = df['reviewCreatedVersion'].fillna('Unknown')
df['appVersion'] = df['appVersion'].fillna('Unknown')


In [ ]:
import re

# 텍스트 전처리 함수 정의
def clean_review(text):
    if pd.isnull(text):
        return ""

    # 1. 한글, 숫자, 공백만 남기기
    text = re.sub(r'[^ㄱ-ㅎ가-힣0-9\s]', '', text)

    # 2. 중복 공백 제거
    text = re.sub(r'\s+', ' ', text).strip()

    return text

# 원본 content 컬럼 보존하고, 전처리된 내용 저장
df['clean_content'] = df['content'].apply(clean_review)

# 결과 확인
print(df[['content', 'clean_content']].head(5))


                                        content  \
0  결제버튼을눌러도 돈도안빠져나가고 예매도안돼요. 돈쓰지말라고 배려해주는건지....   
1                                자동로그인이 자꾸 풀리네요   
2                   오리지널 티켓 위탁점 배포 중단해서 이제 안가요.   
3                             앱 진짜 쓰레기같이 만들어놔ㅛ네   
4              몇시간마다 자동으로 로그아웃이 되니까 너무 불편하네요...   

                             clean_content  
0  결제버튼을눌러도 돈도안빠져나가고 예매도안돼요 돈쓰지말라고 배려해주는건지  
1                           자동로그인이 자꾸 풀리네요  
2               오리지널 티켓 위탁점 배포 중단해서 이제 안가요  
3                         앱 진짜 쓰레기같이 만들어놔네  
4            몇시간마다 자동으로 로그아웃이 되니까 너무 불편하네요  


In [ ]:
import re

# 기본 불용어 리스트 (원하는 단어 추가 가능)
stopwords = ['그리고', '그냥', '정말', '너무', '이거', '저거', '거기', '그런데', '아주', '매우', '좀', '진짜', '때문에', '그때', '근데']

# 텍스트 전처리 함수 (한글 정리 + 불용어 제거)
def clean_and_remove_stopwords(text):
    if pd.isnull(text):
        return ""

    # 1. 한글, 숫자, 공백만 남기기
    text = re.sub(r'[^ㄱ-ㅎ가-힣0-9\s]', '', text)

    # 2. 중복 공백 제거
    text = re.sub(r'\s+', ' ', text).strip()

    # 3. 불용어 제거
    words = text.split()
    words = [word for word in words if word not in stopwords]

    return ' '.join(words)

# 새 컬럼으로 저장
df['clean_no_stopwords'] = df['content'].apply(clean_and_remove_stopwords)

# 전처리 결과 확인
print(df[['content', 'clean_no_stopwords']].head(5))


                                        content  \
0  결제버튼을눌러도 돈도안빠져나가고 예매도안돼요. 돈쓰지말라고 배려해주는건지....   
1                                자동로그인이 자꾸 풀리네요   
2                   오리지널 티켓 위탁점 배포 중단해서 이제 안가요.   
3                             앱 진짜 쓰레기같이 만들어놔ㅛ네   
4              몇시간마다 자동으로 로그아웃이 되니까 너무 불편하네요...   

                        clean_no_stopwords  
0  결제버튼을눌러도 돈도안빠져나가고 예매도안돼요 돈쓰지말라고 배려해주는건지  
1                           자동로그인이 자꾸 풀리네요  
2               오리지널 티켓 위탁점 배포 중단해서 이제 안가요  
3                            앱 쓰레기같이 만들어놔네  
4               몇시간마다 자동으로 로그아웃이 되니까 불편하네요  


In [14]:
import pandas as pd
import re

########################################
# 0) CSV 불러오기 + 컬럼 정리
########################################
file_path = "megabox_reviews.csv"
df = pd.read_csv(file_path)

use_cols = ["userName", "content", "score", "thumbsUpCount", "reviewCreatedVersion", "at"]
df = df[use_cols].copy()

# 날짜 처리(추이/기간 분석용)
df["at_dt"] = pd.to_datetime(df["at"], errors="coerce")
df["date"] = df["at_dt"].dt.date

# 버전 컬럼은 요청대로 reviewCreatedVersion 사용
df["version"] = df["reviewCreatedVersion"].fillna("Unknown")

########################################
# 1) 🚨 Critical Issues (1점 & 공감수 10개 이상)
########################################
critical_df = df[(df["score"] == 1) & (df["thumbsUpCount"] >= 10)].copy()
critical_df = critical_df.sort_values("thumbsUpCount", ascending=False)

# (옵션) Critical Issues에도 간단한 이슈 태그 달기
issue_rules = {
    "결제/예매": [r"결제", r"예매", r"취소", r"환불", r"카드", r"페이", r"오류", r"중지"],
    "로그인/인증": [r"로그인", r"로그아웃", r"본인인증", r"인증", r"회원가입", r"아이디", r"비밀번호"],
    "실행/로딩/튕김": [r"로딩", r"안\s?켜", r"안\s?열", r"멈", r"튕", r"꺼지", r"강제\s?종료", r"렉", r"버벅"],
    "업데이트/버전": [r"업데이트", r"버전", r"이전버전", r"새버전"],
    "광고/UX": [r"광고", r"팝업", r"UI", r"UX", r"인터페이스", r"불편", r"구리", r"짜증"],
    "좌석/쿠폰/티켓": [r"좌석", r"자리", r"쿠폰", r"할인", r"포토카드", r"오리지널\s?티켓", r"티켓", r"이벤트"],
}

def detect_issues(text):
    if pd.isna(text):
        return []
    text = str(text)
    hits = []
    for issue, patterns in issue_rules.items():
        for p in patterns:
            if re.search(p, text, flags=re.IGNORECASE):
                hits.append(issue)
                break
    return hits

critical_df["issues"] = critical_df["content"].apply(detect_issues)
critical_df["primary_issue"] = critical_df["issues"].apply(lambda x: x[0] if x else "기타/미분류")

########################################
# 2) 📉 버전별 앱 안정성 (Version Stats)
# - 버전별: 리뷰수, 평균평점, 1점비율, Critical비율, 안정성 키워드 비율
########################################
stability_patterns = [
    r"오류", r"중지", r"튕", r"꺼지", r"강제\s?종료", r"로딩", r"멈", r"버벅", r"렉",
    r"결제", r"예매", r"로그인", r"로그아웃", r"본인인증", r"인증", r"안\s?됨", r"안\s?돼"
]
stability_re = re.compile("|".join(stability_patterns), flags=re.IGNORECASE)

df["has_stability_kw"] = df["content"].fillna("").astype(str).apply(lambda x: bool(stability_re.search(x)))
df["is_1star"] = (df["score"] == 1).astype(int)
df["is_critical"] = ((df["score"] == 1) & (df["thumbsUpCount"] >= 10)).astype(int)

version_stats_df = (
    df.groupby("version", as_index=False)
      .agg(
          리뷰수=("userName", "count"),
          평균평점=("score", "mean"),
          중앙평점=("score", "median"),
          평균공감수=("thumbsUpCount", "mean"),
          최대공감수=("thumbsUpCount", "max"),
          _1점리뷰수=("is_1star", "sum"),
          크리티컬리뷰수=("is_critical", "sum"),
          안정성키워드포함리뷰수=("has_stability_kw", "sum"),
          최초리뷰일=("at_dt", "min"),
          최종리뷰일=("at_dt", "max"),
      )
)

version_stats_df["1점비율(%)"] = (version_stats_df["_1점리뷰수"] / version_stats_df["리뷰수"] * 100).round(2)
version_stats_df["크리티컬비율(%)"] = (version_stats_df["크리티컬리뷰수"] / version_stats_df["리뷰수"] * 100).round(2)
version_stats_df["안정성키워드비율(%)"] = (version_stats_df["안정성키워드포함리뷰수"] / version_stats_df["리뷰수"] * 100).round(2)

# 작은 표본 버전(리뷰수 적음) 왜곡 방지: 필요하면 컷 설정
# version_stats_df = version_stats_df[version_stats_df["리뷰수"] >= 5].copy()

version_stats_df = version_stats_df.sort_values(["리뷰수", "1점비율(%)"], ascending=[False, False])

########################################
# 3) 🎟️ 이벤트/마케팅 역효과 (Event Issues)
########################################
event_patterns = [
    r"이벤트", r"선착순", r"쿠폰", r"할인", r"빵원", r"0원", r"무료", r"특가",
    r"포토카드", r"오리지널\s?티켓", r"티켓", r"굿즈", r"키링", r"증정", r"경품",
    r"응모", r"스탬프", r"미션", r"프로모션"
]
event_re = re.compile("|".join(event_patterns), flags=re.IGNORECASE)

df["has_event_kw"] = df["content"].fillna("").astype(str).apply(lambda x: bool(event_re.search(x)))
event_df = df[df["has_event_kw"]].copy()

# "역효과"를 보기 쉽게: 낮은 평점 우선 + 공감 높은 순
event_issues_df = event_df.sort_values(["score", "thumbsUpCount"], ascending=[True, False]).copy()

# 이벤트 요약
event_summary_df = pd.DataFrame({
    "이벤트키워드리뷰수": [len(event_df)],
    "이벤트키워드_평균평점": [round(event_df["score"].mean(), 3) if len(event_df) else None],
    "이벤트키워드_1점리뷰수": [int((event_df["score"] == 1).sum())],
    "이벤트키워드_크리티컬리뷰수(1점&공감10+)": [int(((event_df["score"] == 1) & (event_df["thumbsUpCount"] >= 10)).sum())],
})

########################################
# 4) 💎 Hidden Gems (5점 리뷰 속 개선점)
# - 5점이지만 "아쉽/개선/불편/버그/오류/다만/하지만" 등 개선 시그널 포함
########################################
improve_patterns = [
    r"아쉽", r"개선", r"불편", r"버그", r"오류", r"튕", r"로딩", r"렉",
    r"다만", r"하지만", r"그런데", r"만", r"추가해", r"추가되면", r"좋겠", r"바람", r"요청"
]
improve_re = re.compile("|".join(improve_patterns), flags=re.IGNORECASE)

df["has_improve_signal"] = df["content"].fillna("").astype(str).apply(lambda x: bool(improve_re.search(x)))

hidden_gems_df = df[(df["score"] == 5) & (df["has_improve_signal"])].copy()
# 공감수 높은 순으로 정렬(개선점인데 공감까지 받는 케이스)
hidden_gems_df = hidden_gems_df.sort_values("thumbsUpCount", ascending=False)

# Hidden Gems 요약
hidden_gems_summary_df = pd.DataFrame({
    "HiddenGems_리뷰수(5점&개선시그널)": [len(hidden_gems_df)],
    "HiddenGems_평균공감수": [round(hidden_gems_df["thumbsUpCount"].mean(), 3) if len(hidden_gems_df) else None],
})

########################################
# 5) 저장(한글 파일명) + 필요한 컬럼만 저장
########################################
# 요청 컬럼 기준 + 분석용 컬럼 일부만 붙여서 저장
critical_save_cols = use_cols + ["primary_issue", "issues"]
critical_df[critical_save_cols].to_csv("01_크리티컬이슈_1점_공감10이상.csv", index=False, encoding="utf-8-sig")

version_stats_df.to_csv("02_버전별_앱안정성_통계.csv", index=False, encoding="utf-8-sig")

event_issues_df[use_cols].to_csv("이벤트_마케팅_역효과_리뷰목록.csv", index=False, encoding="utf-8-sig")
event_summary_df.to_csv("03_이벤트_마케팅_역효과_요약.csv", index=False, encoding="utf-8-sig")

hidden_gems_df[use_cols].to_csv("04_HiddenGems_5점리뷰_개선점.csv", index=False, encoding="utf-8-sig")
hidden_gems_summary_df.to_csv("04_HiddenGems_요약.csv", index=False, encoding="utf-8-sig")

print("✅ 완료!")
print("1) critical_df:", critical_df.shape)
print("2) version_stats_df:", version_stats_df.shape)
print("3) event_issues_df:", event_issues_df.shape, "| event_summary_df:", event_summary_df.shape)
print("4) hidden_gems_df:", hidden_gems_df.shape, "| hidden_gems_summary_df:", hidden_gems_summary_df.shape)

print("\n✅ 저장 파일:")
print("01_크리티컬이슈_1점_공감10이상.csv")
print("02_버전별_앱안정성_통계.csv")
print("03_이벤트_마케팅_역효과_리뷰목록.csv")
print("03_이벤트_마케팅_역효과_요약.csv")
print("04_HiddenGems_5점리뷰_개선점.csv")
print("04_HiddenGems_요약.csv")


✅ 완료!
1) critical_df: (146, 11)
2) version_stats_df: (131, 14)
3) event_issues_df: (286, 13) | event_summary_df: (1, 4)
4) hidden_gems_df: (100, 14) | hidden_gems_summary_df: (1, 2)

✅ 저장 파일:
01_크리티컬이슈_1점_공감10이상.csv
02_버전별_앱안정성_통계.csv
03_이벤트_마케팅_역효과_리뷰목록.csv
03_이벤트_마케팅_역효과_요약.csv
04_HiddenGems_5점리뷰_개선점.csv
04_HiddenGems_요약.csv
